In [27]:
def select_random_df(dflist):
    random_noun_df = random.choice(dflist)
    return random_noun_df


def get_gender(df):
    possible_genders= ["m","f","n"]
    gender = df.iat[-1,1] # gets the second column of the last row which is where gender is stored
    if gender in possible_genders:
        if gender == "m":
            return "masculine"
        elif gender == "f":
            return "feminine"
        elif gender == "n":
            return "neuter"


def select_noun(noun_df,case=None,plurality=None,verbose=True):
    noun=""

    possible_case_rows = ["nominative","genitive","dative","accusative","vocative"]
    possible_plurality_columns = ["singular","dual","plural"]

    if case not in possible_case_rows:
        print("Invalid case")
        return

    if plurality not in possible_plurality_columns:
        print("Invalid plurality")
        return

    noun_row_index_array =get_indices(noun_df,case)[0] # gets an array containing row indexes
    noun_column_index_array=get_indices(noun_df,plurality)[1] # gets an array of column indexes

    if len(noun_row_index_array)==0 or len(noun_row_index_array)==0:
        print("COULDNT FIND WORD, NUMBER OF TRIES")
        return
        
    # if multiple found then select one at random
    noun_row= random.choice(noun_row_index_array)
    noun_column= random.choice(noun_column_index_array)
    
    noun =strip_accents_and_lowercase(
    noun_df.iat[noun_row,noun_column])
    if verbose:
        print("Selected Noun: "+noun)
    return noun


def select_verb(verb_df,voice=None,mood=None, plurality=None,person=None,verbose=True):
    verb=""

    possible_voice_rows =["active","middle"]
    possible_mood_rows =["indicative","subjunctive","optative","imperative"]

    possible_plurality_columns=["singular","dual","plural"]
    possible_person_columns=["first","second","third"]

    if voice not in possible_voice_rows:
        print("Invalid voice")
        return

    if mood not in possible_mood_rows:
        print("Invalid mood")
        return
    
    if plurality not in  possible_plurality_columns:
        print("Invalid plurality")
        return

    if person not in  possible_person_columns:
        print("Invalid person")
        return
    
    voice_row_index_array = get_indices(verb_df,voice)[0]
    mood_row_index_array = get_indices(verb_df,mood)[0]

    if voice_row_index_array==[''] or mood_row_index_array==['']:
        return ""

    plurality_column_index_array = get_indices(verb_df,plurality)[1]
    person_column_index_array = get_indices(verb_df,person)[1]

    if plurality_column_index_array==[''] or person_column_index_array==['']:
        return ""

    common_indices = set(voice_row_index_array).intersection(mood_row_index_array)
    if common_indices:
        verb_row = random.choice(list(common_indices))
    else:
        return ""
    
    common_indices = set(plurality_column_index_array).intersection(person_column_index_array)
    if common_indices:
        verb_column = random.choice(list(common_indices))
    else:
        return ""
    verb =strip_accents_and_lowercase(
    verb_df.iat[verb_row,verb_column])
    
    if verbose:
        print("Selected Verb: "+verb)
    return verb

   
def select_adjective(adjective_df,case=None,plurality=None,gender=None,verbose=True):
    adjective=""

    possible_case_rows = ["nominative","genitive","dative","accusative","vocative"]
    possible_plurality_columns=["singular","dual","plural"]
    possible_gender_columns=["masculine","feminine","neuter"]

    if case not in possible_case_rows:
        print("Invalid case")
        return

    if plurality not in possible_plurality_columns:
        print("Invalid plurality")
        return
    
    if gender not in  possible_gender_columns:
        print("Invalid gender")
        return


    case_row_index_array =get_indices(adjective_df,case)[0]
    plurality_column_index_array = get_indices(adjective_df,plurality)[1]
    gender_column_index_array = get_indices(adjective_df,gender)[1]

    if plurality_column_index_array==[''] or gender_column_index_array==['']:
        return ""

    adjective_row =random.choice(case_row_index_array)
    adjective_column= random.choice(list(set(plurality_column_index_array).intersection(gender_column_index_array)))
    test_adjective_column=list(set(plurality_column_index_array).intersection(gender_column_index_array))

    #print(plurality_column_index_array)
    #print( gender_column_index_array)
    #print(set(plurality_column_index_array).intersection(gender_column_index_array))
    #print(adjective_column)
    
    adjective =strip_accents_and_lowercase(
    adjective_df.iat[adjective_row,adjective_column])

    if verbose:
        print("Selected Adjective: "+adjective)
    return adjective


def generate_accusative_sentence(noun_dfs,verb_dfs,sentiment,voice,plurality,person,mood,randomize,compound,verbose):
    possible_pluralities=["singular","dual","plural"]
    possible_persons=["first","second","third"]

    if randomize ==True:
        person=random.choice(possible_persons)
        plurality = random.choice(possible_pluralities)

    noun_df = select_random_df(noun_dfs)
    verb_df = select_random_df(verb_dfs)

    noun =select_noun(noun_df,"accusative",plurality,verbose)
    verb =select_verb(verb_df,voice,mood,plurality,person,verbose)
    if verb == "":
        output=[["error","verb not available",""]]
        return output
    if compound:
        noun2 =select_noun(noun_df,"accusative",plurality,verbose)
        verb2 =select_verb(verb_df,voice,mood,plurality,person,verbose)
        conj = strip_accents_and_lowercase('καί')

        output = [[verb,"verb",""],
                  [noun,"noun",sentiment.capitalize()],
                  [conj,"conjugation",""],
                  [verb2,"verb",""],
                  [noun2,"noun",sentiment.capitalize()]]
    else:
        output = [[verb,"verb",""],
                  [noun,"noun",sentiment.capitalize()]]
    return output

    
def generate_nominative_scentence(noun_dfs,verb_dfs,adjective_dfs,sentiment,voice,plurality,person,mood,randomize,compound,verbose):
    possible_pluralities=["singular","dual","plural"]
    possible_persons=["first","second","third"]

    if randomize ==True:
        person=random.choice(possible_persons)
        plurality = random.choice(possible_pluralities)

    noun_df = select_random_df(noun_dfs)
    verb_df = select_random_df(verb_dfs)
    adjective_df = select_random_df(adjective_dfs)

    gender = get_gender(noun_df)
    if gender == None:
        output=[["error","gender not available",""]]
        return output

    noun =select_noun(noun_df,"nominative",plurality,verbose) #nominative

    verb=select_verb(verb_df,voice,mood,plurality,person,verbose)
    if verb == "":
        output=[["error","verb not available",""]]
        return output

    adjective= select_adjective(adjective_df,"nominative",plurality,gender,verbose)

    if compound:
        noun2 =select_noun(noun_df,"nominative",plurality,verbose) #nominative
        verb2=select_verb(verb_df,voice,mood,plurality,person,verbose)
        adjective2= select_adjective(adjective_df,"nominative",plurality,gender,verbose)
        conj = strip_accents_and_lowercase('καί')
        output = [[noun,"noun",sentiment.capitalize()],
                  [verb,"verb",""],
                  [adjective,"adjective",""],
                  [conj,"conjugation",""],
                  [noun2,"noun",sentiment.capitalize()],
                  [verb2,"verb",""],
                  [adjective2,"adjective",""]]
    else:
        output = [[noun,"noun",sentiment.capitalize()],
                  [verb,"verb",""],
                  [adjective,"adjective",""]]
    return output
    

def display_scentnece(scentence):
    for word in scentence:
        display(word[0])


def append_scentence_to_dataset(path,scentence,verbose=True):
    if verbose:
        display(path)
    with open(path,"a",encoding="utf-8") as f:
        f.write("[ O -100\n")
        for word in scentence:
            text = word[0].split(" ")
            for i in range(len(text)):
                if(word[2]!=""):
                    if(i==0):
                        f.write(text[i] +" B-ASP "+word[2]+"\n")
                    else:
                        f.write(text[i] +" I-ASP "+word[2]+"\n")
                else:
                    f.write(text[i] +" O "+"-100\n")
        f.write("] O -100\n\n")
    return


def test_create_accusative_scentence(sentiment="positive",voice="active",plurality="singular",
                                     person="first",mood="indicative",randomize=True,compound=False,verbose=True):
    matched_nouns= match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns",verbose=verbose)
    nouns=get_each_df_from_path_list(matched_nouns)
    scentence = None
    if (sentiment == "positive"):
        matched_verbs_pos= match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/actionverbs/positive",file_contains="present",verbose=verbose)
        verbs= get_each_df_from_path_list(matched_verbs_pos)   

    elif (sentiment == "negative"):
        matched_verbs_neg= match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/actionverbs/negative",file_contains="present",verbose=verbose)
        verbs= get_each_df_from_path_list(matched_verbs_neg) 
    else:
        raise Exception("Please input valid sentiment parameter.")
    
    scentence=generate_accusative_sentence(nouns,verbs,sentiment,voice,plurality,person,mood,randomize,compound,verbose) 
    if verbose:
        display(scentence)
    if scentence[0][0] != 'error':
        append_scentence_to_dataset("greek_output/Intermediate/accusative.atepc",scentence,verbose)
        print("Success")
    else:
        print("Failed")
    return

    
def test_create_nominative_scentence(sentiment="positive",voice="active",plurality="singular",
                                     person="first",mood="indicative",randomize=True,compound=False,verbose=True):
    matched_nouns= match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns",verbose=verbose)
    matched_verbs= match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/linkingverbs/",verbose=verbose)
    nouns=get_each_df_from_path_list(matched_nouns)
    verbs= get_each_df_from_path_list(matched_verbs)
    
    scentence=[[]]
    if (sentiment == "positive"):
        matched_adj_pos= match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/positive",verbose=verbose)
        adjectives=get_each_df_from_path_list(matched_adj_pos)
    elif(sentiment == "negative"):
        matched_adj_neg= match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/negative",verbose=verbose)
        adjectives=get_each_df_from_path_list(matched_adj_neg)
    else:
        raise Exception("Please input valid sentiment parameter.")
    
    scentence =generate_nominative_scentence(nouns,verbs,adjectives,sentiment,voice,plurality,person,mood,randomize,compound,verbose)
    if verbose:
        display(scentence)
    if scentence[0][0] != 'error':
        append_scentence_to_dataset("greek_output/Intermediate/nominative.atepc",scentence,verbose)
        print("Success")
    else:
        print("Failed")
    
    return